# [Advent of Code 2020: Day 3](https://adventofcode.com/2020/day/3)

## \-\-\- Day 3: Toboggan Trajectory \-\-\-

With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (`.`) and trees (`#`) you can see. For example:

```
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#

```

These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:

```
..##.........##.........##.........##.........##.........##.......  --->
#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.##.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........#.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...##....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  --->

```

You start on the open square (`.`) in the top\-left corner and need to reach the bottom (below the bottom\-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by **counting all the trees** you would encounter for the slope **right 3, down 1**:

From your starting position at the top\-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

The locations you'd check in the above example are marked here with **`O`** where there was an open square and **`X`** where there was a tree:

```
..##.........##.........##.........##.........##.........##.......  --->
#..O#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....X..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#O#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..X...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.X#.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#.O..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........X.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.X#...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...#X....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...X.#.#..#...#.#.#..#...#.#.#..#...#.#  --->

```

In this example, traversing the map using this slope would cause you to encounter **`7`** trees.

Starting at the top\-left corner of your map and following a slope of right 3 and down 1, **how many trees would you encounter?**

In [1]:
import unittest
from IPython.display import Markdown, display

from aoc_puzzle import AocPuzzle

class TreeMap(AocPuzzle):
    
    EMPTY = '.'
    TREE = '#'
    NEWLINE = "\n"
    start_coord = (0,0)
        
    def parse_data(self, raw_data):
        x = 0
        y = 0
        self.data = []
        for point in raw_data:
            if point is self.TREE:
                self.data.append((x,y))
                
            if point is self.NEWLINE:
                y += 1
                x = 0
            else:
                x +=1
        
        self.width = x + 1
        self.height = y + 1
    
    def incriment_coord(self, coord, step):
        x,y = coord
        xstep, ystep = step
        return (x+xstep, y+ystep)
    
    def is_tree(self, coord):
        """ Does a tree exist at the given coordinate """
        x,y = coord
        
        # Since the tree pattern repeats on the x axis
        # we must ensure x will always be a valid value
        x = x % (self.width - 1)
        
        if y > self.height:
            raise Exception(f'Coordinate {coord} is off grid')
            
        return ((x,y) in self.data)
    
    def count_trees(self, slope, output=False):
        coord = self.start_coord
        count = 0
        
        while coord[1] < self.height:
            if coord[0] != 0:
                if self.is_tree(coord):
                    count += 1
                        
            coord = self.incriment_coord(coord, slope)
        
        if output:
            display(Markdown(f'#### Encountered `{count}` trees at a slope of `{slope}`' ))
        
        return count


class TestBasic(unittest.TestCase):
    
    def test_load_data(self):
        in_data = ".#..#\n.....\n#####\n....#\n...##"
        exp_out = [(1,0),(4,0),(0,2),(1,2),(2,2),(3,2),(4,2),(4,3),(3,4),(4,4)]
        tm = TreeMap(in_data)
        self.assertEqual(tm.data, exp_out)
        
    def test_is_tree(self):
        in_data = ".#..#\n.....\n#####\n....#\n...##"
        coords = [(1,0),(0,1),(6,0)]
        exp_output = [True,False,True]
        tm = TreeMap(in_data)
        for coord, exp_out in tuple(zip(coords, exp_output)):
            self.assertEqual(tm.is_tree(coord), exp_out)
            
    def test_count_trees(self):
        in_data = '..##.......\n#...#...#..\n.#....#..#.\n..#.#...#.#\n.#...##..#.\n..#.##.....\n.#.#.#....#\n.#........#\n#.##...#...\n#...##....#\n.#..#...#.#'
        exp_out = 7
        tm = TreeMap(in_data)
        self.assertEqual(tm.count_trees((3,1)), exp_out)        
        
unittest.main(argv=[""], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


In [2]:
tm = TreeMap("input/d03.txt")
tm.count_trees((3,1), output=True)

#### Encountered `148` trees at a slope of `(3, 1)`

148

## \-\-\- Part Two \-\-\-

Time to check the rest of the slopes \- you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top\-left corner and traverse the map all the way to the bottom:

*   Right 1, down 1.
*   Right 3, down 1. (This is the slope you already checked.)
*   Right 5, down 1.
*   Right 7, down 1.
*   Right 1, down 2.

In the above example, these slopes would find `2`, `7`, `3`, `4`, and `2` tree(s) respectively; multiplied together, these produce the answer **`336`**.

**What do you get if you multiply together the number of trees encountered on each of the listed slopes?**

In [3]:
class TestBasic(unittest.TestCase):

    def test_count_trees(self):
        map_data = '..##.......\n#...#...#..\n.#....#..#.\n..#.#...#.#\n.#...##..#.\n..#.##.....\n.#.#.#....#\n.#........#\n#.##...#...\n#...##....#\n.#..#...#.#'
        test_data = [((1,1),2),((3,1),7),((5,1),3),((7,1),4),((1,2),2)]
        tm = TreeMap(map_data)
        for slope, ans in test_data:
            self.assertEqual(tm.count_trees(slope),ans)
        
        
unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [4]:
tm = TreeMap("input/d03.txt")

slope_list = [(1,1),(3,1),(5,1),(7,1),(1,2)]

tree_count_list = []
for slope in slope_list:
    tree_count_list.append(tm.count_trees(slope, output=True))
    
product = 1
for tree_count in tree_count_list:
    product *= tree_count
display(Markdown(f'### Product of all trees encountered: `{product}`'))

#### Encountered `50` trees at a slope of `(1, 1)`

#### Encountered `148` trees at a slope of `(3, 1)`

#### Encountered `53` trees at a slope of `(5, 1)`

#### Encountered `64` trees at a slope of `(7, 1)`

#### Encountered `29` trees at a slope of `(1, 2)`

### Product of all trees encountered: `727923200`